## LOADING THE LIBRARIES

In [7]:
import numpy as np
import pandas as pd
# for splitting the train data
from sklearn.model_selection import train_test_split as splt
# for feature scaling
from sklearn.preprocessing import StandardScaler
# importing the keras library for ANN
# sequential module required to initialize the ANN
from tensorflow.keras.models import Sequential
# dense module required to build layers in the ANN model
from tensorflow.keras.layers import Dense
# for confusion matrix
from sklearn.metrics import confusion_matrix
# importing keras wrapper or classifier
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# for k-fold cross validation
from sklearn.model_selection import cross_val_score
# for implementing dropout regularization
from tensorflow.keras.layers import Dropout
# for parameter tuning with grid search
from sklearn.model_selection import GridSearchCV

## Part 1 - Data Preprocessing
- **Importing The dataset**

In [8]:
dataset = pd.read_csv('Churn_Modelling.csv')

In [9]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


- **Separating The Independent Variable**

In [10]:
X = dataset.drop('RowNumber,CustomerId,Surname,Exited'.split(','),axis=1)
Y = dataset['Exited']

In [11]:
print(X.head(1))
print(Y.head(1))

   CreditScore Geography  Gender  Age  Tenure  Balance  NumOfProducts  \
0          619    France  Female   42       2      0.0              1   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               1        101348.88  
0    1
Name: Exited, dtype: int64


- **Creating The Dummy Variables**

In [12]:
dataset = X
for i in 'Geography,Gender'.split(','):
    data = pd.get_dummies(dataset[i],drop_first=True)
    dataset = pd.concat([dataset,data],axis=1)

In [13]:
X = dataset

In [14]:
X.drop('Geography,Gender'.split(','),axis=1,inplace=True)

In [15]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
CreditScore        10000 non-null int64
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Germany            10000 non-null uint8
Spain              10000 non-null uint8
Male               10000 non-null uint8
dtypes: float64(2), int64(6), uint8(3)
memory usage: 654.4 KB


In [16]:
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


- **Splitting The Dataset Into Training And Test Set**

In [17]:
# from sklearn.model_selection import train_test_split as splt

In [18]:
X_train, X_test, Y_train, Y_test = splt(X, Y, test_size=0.2, random_state=0)

In [19]:
print(X_train.info())
print(X_test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8000 entries, 7389 to 2732
Data columns (total 11 columns):
CreditScore        8000 non-null int64
Age                8000 non-null int64
Tenure             8000 non-null int64
Balance            8000 non-null float64
NumOfProducts      8000 non-null int64
HasCrCard          8000 non-null int64
IsActiveMember     8000 non-null int64
EstimatedSalary    8000 non-null float64
Germany            8000 non-null uint8
Spain              8000 non-null uint8
Male               8000 non-null uint8
dtypes: float64(2), int64(6), uint8(3)
memory usage: 585.9 KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 9394 to 6690
Data columns (total 11 columns):
CreditScore        2000 non-null int64
Age                2000 non-null int64
Tenure             2000 non-null int64
Balance            2000 non-null float64
NumOfProducts      2000 non-null int64
HasCrCard          2000 non-null int64
IsActiveMember     2000 non-null int64
Estim

In [20]:
Y_train = pd.DataFrame(Y_train,columns=['Exited'])
Y_test = pd.DataFrame(Y_test,columns=['Exited'])

In [21]:
print(Y_train.head())
print(Y_test.head())

      Exited
7389       0
9275       0
2995       0
5316       0
356        0
      Exited
9394       0
898        1
2398       0
5906       0
2343       0


- **Feature Scaling**

In [22]:
# from sklearn.preprocessing import StandardScaler

In [23]:
scale = StandardScaler()

In [24]:
data = X_train
# fitting and transforming the X_train into feature scaling
X_train = scale.fit_transform(X_train)
# converting again into dataframe
X_train = pd.DataFrame(X_train,columns=data.columns)
data = X_test
# fitting and transforming the X_test into feature scaling
X_test = scale.transform(X_test)
# converting again into dataframe
X_test = pd.DataFrame(X_test,columns=data.columns)

In [25]:
X_train.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,0.169582,-0.464608,0.006661,-1.215717,0.809503,0.642595,-1.032270,1.106432,-0.569844,1.743090,-1.091687
1,-2.304559,0.301026,-1.377440,-0.006312,-0.921591,0.642595,0.968738,-0.748664,1.754865,-0.573694,0.916013
2,-1.191196,-0.943129,-1.031415,0.579935,-0.921591,0.642595,-1.032270,1.485335,-0.569844,-0.573694,-1.091687
3,0.035566,0.109617,0.006661,0.473128,-0.921591,0.642595,-1.032270,1.276528,-0.569844,1.743090,0.916013
4,2.056114,1.736588,1.044737,0.810193,0.809503,0.642595,0.968738,0.558378,-0.569844,1.743090,-1.091687


In [26]:
X_test.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,-0.552043,-0.368904,1.044737,0.879303,-0.921591,0.642595,0.968738,1.610857,1.754865,-0.573694,-1.091687
1,-1.314903,0.109617,-1.031415,0.429722,-0.921591,0.642595,-1.032270,0.495870,-0.569844,-0.573694,-1.091687
2,0.571630,0.301026,1.044737,0.308583,-0.921591,0.642595,0.968738,-0.424787,-0.569844,1.743090,-1.091687
3,1.416961,-0.656016,-0.339364,0.575336,-0.921591,-1.556190,-1.032270,-0.187777,-0.569844,-0.573694,0.916013
4,0.571630,-0.081791,0.006661,1.389611,0.809503,0.642595,0.968738,0.616842,1.754865,-0.573694,0.916013


## Part 2 - Building The ANN Model

In [ ]:
# import keras
# from tensorflow.keras.models import Sequential 
# from tensorflow.keras.layers import Dense

- **Initialising The ANN**

In [ ]:
classifier = Sequential()

- **Adding The Input Layer And The First Hidden Layer**

In [ ]:
classifier.add(Dense(units=6,kernel_initializer='uniform',
                     activation='relu',input_dim=11))

- **Adding The Second Hidden Layer**

In [ ]:
classifier.add(Dense(units=6,kernel_initializer='uniform',
                     activation='relu'))

- **Adding The Output Layer**

In [ ]:
classifier.add(Dense(units=1,kernel_initializer='uniform',
                     activation='sigmoid'))
# if the dependent variable has more than two categories,
# for eg: 3 categories, you have to apply units=3 and activition='softmax'

- **Compiling The ANN**

In [ ]:
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
# if the dependent variable has more than two variables say three,
# then loss= 'categorical_crossentropy'

- **Fitting The ANN To The Training Set**

In [ ]:
classifier.fit(x=X_train,y=Y_train,batch_size=10,epochs=100)

## Part 3 - Making The Predictions And Evaluating The Model

- **Predicting The Test Set Results**

In [ ]:
Y_pred = classifier.predict(X_test)

In [ ]:
Y_pred

In [ ]:
Y_pred = (Y_pred > 0.5)

In [ ]:
pd.DataFrame(Y_pred,columns=['Predicted'])

- **Making The Confusion Matrix**

In [ ]:
# from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test,Y_pred)

In [ ]:
cm

## Part 4 - Homework

### Predict if the customer with the following informations will leave the bank:
- **Geography: France**
- **Credit Score: 600**
- **Gender: Male**
- **Age: 40**
- **Tenure: 3**
- **Balance: 60000**
- **Number Of Products: 2**
- **Has Credit Card: yes**
- **Is Active Member: Yes**
- **Estimated Salary: 50000**

In [ ]:
X_train.head(1)

In [ ]:
'600,40,3,60000,2,1,1,50000,0,0,1'.split(',')

In [ ]:
new_pred = classifier.predict(scale.transform(np.array([[600, 40, 3, 60000, 2, 1, 1, 50000, 0, 0, 1]])))
new_pred = (new_pred > 0.5)

In [ ]:
new_pred

## Part 5 - Evaluating, Improving And Tuning The ANN

- **5.1. Evaluating The ANN**

In [ ]:
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# from sklearn.model_selection import cross_val_score

In [27]:
# making function that build the architecture of the classifier
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units=6,kernel_initializer='uniform',
                     activation='relu',input_dim=11))
    classifier.add(Dense(units=6,kernel_initializer='uniform',
                     activation='relu'))
    classifier.add(Dense(units=1,kernel_initializer='uniform',
                     activation='sigmoid'))
    classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn= build_classifier, batch_size=10, epochs=100)

In [28]:
accuracies = cross_val_score(estimator=classifier, X=X_train, y=Y_train, cv=8, n_jobs= -1)

In [29]:
accuracies

array([0.852     , 0.833     , 0.82999998, 0.84399998, 0.85399997,
       0.77999997, 0.84500003, 0.85799998])

In [30]:
mean = accuracies.mean()
print(mean)
variance = accuracies.std()
print(variance)

0.8369999900460243
0.023404064407480853


In [31]:
Y_pred = accuracies.predict(X_test)

AttributeError: 'numpy.ndarray' object has no attribute 'predict'

- **5.2. Improving The ANN**
  - **Dropout Regularization To Reduce Overfitting If Needed**

In [20]:
# from tensorflow.keras.layers import Dropout

- **Initialising The ANN**

In [ ]:
classifier = Sequential()

- **Adding The Input Layer And The First Hidden Layer With Dropout**

In [ ]:
classifier.add(Dense(units=6,kernel_initializer='uniform',
                     activation='relu',input_dim=11))
classifier.add(Dropout(rate=0.1))

- **Adding The Second Hidden Layer**

In [ ]:
classifier.add(Dense(units=6,kernel_initializer='uniform',
                     activation='relu'))
classifier.add(Dropout(rate=0.1))

- **Adding The Output Layer**

In [ ]:
classifier.add(Dense(units=1,kernel_initializer='uniform',
                     activation='sigmoid'))
# if the dependent variable has more than two categories,
# for eg: 3 categories, you have to apply units=3 and activition='softmax'

- **Compiling The ANN**

In [ ]:
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
# if the dependent variable has more than two variables say three,
# then loss= 'categorical_crossentropy'

- **5.3. Tuning The ANN**

In [ ]:
# from sklearn.model_selection import GridSearchCV

In [22]:
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units=6,kernel_initializer='uniform',
                     activation='relu',input_dim=11))
    classifier.add(Dense(units=6,kernel_initializer='uniform',
                     activation='relu'))
    classifier.add(Dense(units=1,kernel_initializer='uniform',
                     activation='sigmoid'))
    classifier.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn= build_classifier)

In [23]:
parameters = {'batch_size': [25,32],
             'epochs': [100,500],
             'optimizer': ['adam','rmsprop']}

In [29]:
grid_search = GridSearchCV(estimator=classifier,
                          param_grid=parameters,
                          scoring='accuracy',
                          cv=8)

In [ ]:
grid_search = grid_search.fit(X=X_train,y=Y_train)

Train on 7000 samples
Epoch 1/100
7000/7000 [==============================] - 4s 575us/sample - loss: 0.5786 - accuracy: 0.7970
Epoch 2/100
7000/7000 [==============================] - 2s 278us/sample - loss: 0.4349 - accuracy: 0.7977
Epoch 3/100
7000/7000 [==============================] - 2s 290us/sample - loss: 0.4291 - accuracy: 0.7977
Epoch 4/100
7000/7000 [==============================] - 2s 319us/sample - loss: 0.4263 - accuracy: 0.7977
Epoch 5/100
7000/7000 [==============================] - 2s 341us/sample - loss: 0.4242 - accuracy: 0.7977
Epoch 6/100
7000/7000 [==============================] - 2s 349us/sample - loss: 0.4220 - accuracy: 0.7977
Epoch 7/100
7000/7000 [==============================] - 2s 291us/sample - loss: 0.4203 - accuracy: 0.7977
Epoch 8/100
7000/7000 [==============================] - 2s 355us/sample - loss: 0.4183 - accuracy: 0.8016
Epoch 9/100
7000/7000 [==============================] - 2s 337us/sample - loss: 0.4169 - accuracy: 0.8234
Epoch 10/100
70

7000/7000 [==============================] - 1s 208us/sample - loss: 0.5054 - accuracy: 0.7964
Epoch 48/100
7000/7000 [==============================] - 1s 214us/sample - loss: 0.5054 - accuracy: 0.7964
Epoch 49/100
7000/7000 [==============================] - 1s 214us/sample - loss: 0.5054 - accuracy: 0.7964
Epoch 50/100
7000/7000 [==============================] - 2s 227us/sample - loss: 0.5054 - accuracy: 0.7964
Epoch 51/100
7000/7000 [==============================] - 2s 233us/sample - loss: 0.5054 - accuracy: 0.7964
Epoch 52/100
7000/7000 [==============================] - 1s 193us/sample - loss: 0.5053 - accuracy: 0.7964
Epoch 53/100
7000/7000 [==============================] - 1s 203us/sample - loss: 0.5053 - accuracy: 0.7964
Epoch 54/100
7000/7000 [==============================] - 1s 202us/sample - loss: 0.5054 - accuracy: 0.7964
Epoch 55/100
7000/7000 [==============================] - 1s 212us/sample - loss: 0.5053 - accuracy: 0.7964
Epoch 56/100
7000/7000 [=================

7000/7000 [==============================] - 2s 227us/sample - loss: 0.3998 - accuracy: 0.8354
Epoch 95/100
7000/7000 [==============================] - 1s 198us/sample - loss: 0.3995 - accuracy: 0.8361
Epoch 96/100
7000/7000 [==============================] - 1s 198us/sample - loss: 0.4001 - accuracy: 0.8356
Epoch 97/100
7000/7000 [==============================] - 2s 225us/sample - loss: 0.3995 - accuracy: 0.8380
Epoch 98/100
7000/7000 [==============================] - 2s 254us/sample - loss: 0.3999 - accuracy: 0.8341
Epoch 99/100
7000/7000 [==============================] - 1s 207us/sample - loss: 0.3994 - accuracy: 0.8361
Epoch 100/100
7000/7000 [==============================] - 1s 206us/sample - loss: 0.3997 - accuracy: 0.8360
Train on 7000 samples
Epoch 1/100
7000/7000 [==============================] - 2s 291us/sample - loss: 0.5689 - accuracy: 0.7941
Epoch 2/100
7000/7000 [==============================] - 2s 231us/sample - loss: 0.4395 - accuracy: 0.7941
Epoch 3/100
7000/700

7000/7000 [==============================] - 1s 213us/sample - loss: 0.3963 - accuracy: 0.8350
Epoch 39/100
7000/7000 [==============================] - 1s 207us/sample - loss: 0.3961 - accuracy: 0.8341
Epoch 40/100
7000/7000 [==============================] - 2s 231us/sample - loss: 0.3957 - accuracy: 0.8351
Epoch 41/100
7000/7000 [==============================] - 1s 211us/sample - loss: 0.3956 - accuracy: 0.8344
Epoch 42/100
7000/7000 [==============================] - 1s 208us/sample - loss: 0.3957 - accuracy: 0.8349
Epoch 43/100
7000/7000 [==============================] - 1s 206us/sample - loss: 0.3956 - accuracy: 0.8350
Epoch 44/100
7000/7000 [==============================] - 1s 206us/sample - loss: 0.3952 - accuracy: 0.8341
Epoch 45/100
7000/7000 [==============================] - 2s 229us/sample - loss: 0.3953 - accuracy: 0.8353
Epoch 46/100
7000/7000 [==============================] - 2s 215us/sample - loss: 0.3953 - accuracy: 0.8350
Epoch 47/100
7000/7000 [=================

Epoch 86/100
7000/7000 [==============================] - 2s 226us/sample - loss: 0.3968 - accuracy: 0.8361
Epoch 87/100
7000/7000 [==============================] - 1s 200us/sample - loss: 0.3969 - accuracy: 0.8391
Epoch 88/100
7000/7000 [==============================] - 1s 206us/sample - loss: 0.3970 - accuracy: 0.8380
Epoch 89/100
7000/7000 [==============================] - 2s 222us/sample - loss: 0.3970 - accuracy: 0.8367
Epoch 90/100
7000/7000 [==============================] - 2s 241us/sample - loss: 0.3968 - accuracy: 0.8374
Epoch 91/100
7000/7000 [==============================] - 1s 204us/sample - loss: 0.3970 - accuracy: 0.8391
Epoch 92/100
7000/7000 [==============================] - 1s 205us/sample - loss: 0.3966 - accuracy: 0.8371
Epoch 93/100
7000/7000 [==============================] - 2s 221us/sample - loss: 0.3965 - accuracy: 0.8393
Epoch 94/100
7000/7000 [==============================] - 2s 238us/sample - loss: 0.3964 - accuracy: 0.8386
Epoch 95/100
7000/7000 [====

...

7000/7000 [==============================] - 2s 237us/sample - loss: 0.4077 - accuracy: 0.8364
Epoch 22/100
7000/7000 [==============================] - 2s 225us/sample - loss: 0.4073 - accuracy: 0.8360
Epoch 23/100
7000/7000 [==============================] - 1s 206us/sample - loss: 0.4073 - accuracy: 0.8347
Epoch 24/100
7000/7000 [==============================] - 2s 228us/sample - loss: 0.4067 - accuracy: 0.8366
Epoch 25/100
7000/7000 [==============================] - 1s 201us/sample - loss: 0.4066 - accuracy: 0.8363
Epoch 26/100
7000/7000 [==============================] - 2s 215us/sample - loss: 0.4057 - accuracy: 0.8344
Epoch 27/100
7000/7000 [==============================] - 1s 214us/sample - loss: 0.4057 - accuracy: 0.8356
Epoch 28/100
7000/7000 [==============================] - 1s 202us/sample - loss: 0.4055 - accuracy: 0.8364
Epoch 29/100
7000/7000 [==============================] - 2s 241us/sample - loss: 0.4052 - accuracy: 0.8353
Epoch 30/100
7000/7000 [=================

Epoch 97/100
7000/7000 [==============================] - 2s 310us/sample - loss: 0.4001 - accuracy: 0.8360
Epoch 98/100
7000/7000 [==============================] - 2s 259us/sample - loss: 0.4005 - accuracy: 0.8340
Epoch 99/100
7000/7000 [==============================] - 2s 341us/sample - loss: 0.4000 - accuracy: 0.8357
Epoch 100/100
7000/7000 [==============================] - 2s 319us/sample - loss: 0.4003 - accuracy: 0.8364
Train on 7000 samples
Epoch 1/100
7000/7000 [==============================] - 3s 405us/sample - loss: 0.6045 - accuracy: 0.7924
Epoch 2/100
7000/7000 [==============================] - 2s 329us/sample - loss: 0.4580 - accuracy: 0.7941
Epoch 3/100
7000/7000 [==============================] - 2s 263us/sample - loss: 0.4398 - accuracy: 0.7941
Epoch 4/100
7000/7000 [==============================] - 2s 235us/sample - loss: 0.4354 - accuracy: 0.7941
Epoch 5/100
7000/7000 [==============================] - 2s 229us/sample - loss: 0.4330 - accuracy: 0.7941
Epoch 6/10

7000/7000 [==============================] - 2s 252us/sample - loss: 0.3952 - accuracy: 0.8381
Epoch 42/100
7000/7000 [==============================] - 2s 239us/sample - loss: 0.3950 - accuracy: 0.8370
Epoch 43/100
7000/7000 [==============================] - 2s 257us/sample - loss: 0.3953 - accuracy: 0.8383
Epoch 44/100
7000/7000 [==============================] - 2s 237us/sample - loss: 0.3950 - accuracy: 0.8384
Epoch 45/100
7000/7000 [==============================] - 2s 224us/sample - loss: 0.3946 - accuracy: 0.8353
Epoch 46/100
7000/7000 [==============================] - 2s 266us/sample - loss: 0.3947 - accuracy: 0.8400
Epoch 47/100
7000/7000 [==============================] - 2s 237us/sample - loss: 0.3948 - accuracy: 0.8384
Epoch 48/100
7000/7000 [==============================] - 2s 228us/sample - loss: 0.3946 - accuracy: 0.8389
Epoch 49/100
7000/7000 [==============================] - 2s 232us/sample - loss: 0.3947 - accuracy: 0.8384
Epoch 50/100
7000/7000 [=================

Epoch 89/100
7000/7000 [==============================] - 2s 264us/sample - loss: 0.3916 - accuracy: 0.8374
Epoch 90/100
7000/7000 [==============================] - 2s 234us/sample - loss: 0.3913 - accuracy: 0.8380
Epoch 91/100
7000/7000 [==============================] - 2s 234us/sample - loss: 0.3913 - accuracy: 0.8389
Epoch 92/100
7000/7000 [==============================] - 2s 250us/sample - loss: 0.3913 - accuracy: 0.8371
Epoch 93/100
7000/7000 [==============================] - 2s 246us/sample - loss: 0.3915 - accuracy: 0.8386
Epoch 94/100
7000/7000 [==============================] - 2s 235us/sample - loss: 0.3914 - accuracy: 0.8384
Epoch 95/100
7000/7000 [==============================] - 2s 276us/sample - loss: 0.3913 - accuracy: 0.8381
Epoch 96/100
7000/7000 [==============================] - 2s 246us/sample - loss: 0.3913 - accuracy: 0.8376
Epoch 97/100
7000/7000 [==============================] - ETA: 0s - loss: 0.3925 - accuracy: 0.83 - 1s 209us/sample - loss: 0.3914 - acc

7000/7000 [==============================] - 3s 376us/sample - loss: 0.3978 - accuracy: 0.8370
Epoch 33/100
7000/7000 [==============================] - 4s 508us/sample - loss: 0.3975 - accuracy: 0.8351
Epoch 34/100
7000/7000 [==============================] - 3s 470us/sample - loss: 0.3972 - accuracy: 0.8374
Epoch 35/100
7000/7000 [==============================] - 3s 438us/sample - loss: 0.3970 - accuracy: 0.8360
Epoch 36/100
7000/7000 [==============================] - 2s 266us/sample - loss: 0.3968 - accuracy: 0.8361
Epoch 37/100
7000/7000 [==============================] - 4s 558us/sample - loss: 0.3968 - accuracy: 0.8359
Epoch 38/100
7000/7000 [==============================] - 3s 397us/sample - loss: 0.3968 - accuracy: 0.8361
Epoch 39/100
7000/7000 [==============================] - 3s 487us/sample - loss: 0.3966 - accuracy: 0.8357
Epoch 40/100
7000/7000 [==============================] - 3s 445us/sample - loss: 0.3965 - accuracy: 0.8364
Epoch 41/100
7000/7000 [=================

In [ ]:
best_parameters = grid_search.best_parameters

In [ ]:
best_accuracy = grid_search.best_accuracy